In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from torch import nn, optim

from nnbma.networks import DenselyConnected, PolynomialNetwork, FullyConnected
from nnbma.dataset import RegressionDataset, MaskDataset
from nnbma.learning import learning_procedure, LearningParameters, MaskedMSELoss, LinearBatchScheduler, BatchScheduler

from nnbma.layers import PolynomialExpansion

np.random.seed(42)


In [ ]:

def error_factor (y_hat: np.ndarray, y: np.ndarray) :
     ef_vector=np.exp(np.log(10)*np.abs(y_hat - y))
     return (np.percentile(ef_vector,99),np.mean(ef_vector))

def metric(y_hat: np.ndarray, y: np.ndarray):
    return np.mean((y_hat - y) ** 2)


In [ ]:
# folder in which the trained neural net will be saved
output_root = "../data/models"

# folder containing the dataset of evaluations of the astrophysical simulator
data_dir = "../data/"

In [ ]:
df_dataset = pd.read_csv(f"{data_dir}/dataset_train_test.csv", index_col=0)

idx = np.arange(len(df_dataset))
np.random.shuffle(idx)

train_frac = 0.7
idx_max_train = int(0.7 * len(df_dataset))

df_train = df_dataset.iloc[idx[:idx_max_train], :] * 1
df_test = df_dataset.iloc[idx[idx_max_train:], :] * 1

X_train = np.log10(df_train.iloc[:, :3].values)
Y_train = np.log10(df_train.iloc[:, 3:].values)
X_test = np.log10(df_test.iloc[:, :3].values)
Y_test = np.log10(df_test.iloc[:, 3:].values)

X_labels = list(df_train.columns)[:3]
Y_labels = list(df_train.columns)[3:]

X_labels, Y_labels

In [ ]:
L = 3

In [ ]:
Y_train

In [ ]:
#%%  Normalisation des données entre 0 et 1

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


train_dataset = RegressionDataset(X_train, Y_train)
test_dataset = RegressionDataset(X_test, Y_test)
print(f"Number of training entries: {X_train.shape[0]:,}") # 154
print(f"Number of testing entries: {X_test.shape[0]:,}") #20 #49

n_layers = 4 # controls the number of layers in your neural network
growing_factor = 0.5

activation = nn.GELU()

In [ ]:
n_features = X_train.shape[1] * 1 # input vector size

# -------------Polynomial Expansion
order = 3  # no larger, otherwise will overfit
n_poly = PolynomialExpansion.expanded_features(order, n_features)

# -------------Densely connected network
subnet = DenselyConnected(n_poly, L, n_layers, growing_factor, activation, outputs_names=Y_labels)

# -------------Combining both
net = PolynomialNetwork(
    n_features,
    order,
    subnet,
    inputs_names=X_labels,
    outputs_names=Y_labels
)

net.poly.update_standardization(x=train_dataset.x, reset=True)

In [ ]:
# display the neural network architecture
net

In [ ]:
# display the number of parameters to train in the neural network
net.count_parameters()

In [ ]:
epochs = 50
batch_size = len(df_train)

loss = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.AdamW(net.parameters(), learning_rate)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=20, factor=0.9
)

learning_params = LearningParameters(
    loss, epochs, batch_size, optimizer, scheduler
)

results = learning_procedure(
    net,
    (train_dataset, test_dataset),
    learning_params,
    val_frac=None,
)

print(f"Loss over training set: {metric(net(X_train), Y_train):.2e}")
print(f"Loss over testing set: {metric(net(X_test), Y_test):.2e}")


In [ ]:
# Evaluate the trained neural network
# For a definition of the error factor, see the nnbma paper (https://www.aanda.org/articles/aa/abs/2023/10/aa47074-23/aa47074-23.html), equation 6.

# for train and test, two values are shown:
# the first is the 99th percentile, ie, a robust estimation of the max
# the second is the mean
print(f"Error Factor over training set: {error_factor(net(X_train), Y_train)}")
print(f"Error Factor over testing set: {error_factor(net(X_test), Y_test)}")

In [ ]:
skip_first = 20

plt.figure(figsize=(1.2 * 6.4, 0.6 * 4.8))

plt.subplot(1, 2, 1)
plt.semilogy(results["train_loss"][skip_first:],"--",label="Train loss")
plt.semilogy(results["val_loss"][skip_first:], label="Test loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title(f"loss evolution (skipping the first {skip_first} iterations)")
plt.legend()

plt.subplot(1, 2, 2)
plt.semilogy(results["lr"][skip_first:],"--")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.title("learning rate evolution")

plt.tight_layout()
plt.show()


In [ ]:
name = "model" # name of the folder in which the neural network is saved
net.save(name, output_root)

import pickle
with open(f"{output_root}/{name}/scaler.pickle", 'wb') as handle:
    pickle.dump(scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)